In [51]:
import tensorflow as tf
import numpy as np

from tqdm import tqdm

In [52]:
import sys, os
from pathlib import Path

In [53]:
sys.path.append('../')

In [54]:
from src.utils import paths_manager as pathsman

In [55]:
selected_profile = pathsman.PROFILES_DIR / "unetnilm_ukdale_20240321_155419"

print(f"Selected profile {selected_profile.resolve()} {'exists' if selected_profile.exists() else 'does not exist'}")

Selected profile C:\Users\MTinaco\Dev\Solutions\cos-algo-nilm\profiles\unetnilm_ukdale_20240321_155419 exists


In [56]:
denoise_inputs = np.load(selected_profile / "training" / "denoise_inputs.npy")
noise_inputs = np.load(selected_profile / "training" / "noise_inputs.npy")
targets = np.load(selected_profile / "training" / "targets.npy")
states = np.load(selected_profile / "training" / "states.npy")

In [57]:
def split_data(data):
	split_1 = int(0.60 * len(data))
	split_2 = int(0.85 * len(data))
	train = data[:split_1]
	validation = data[split_1:split_2]
	test = data[split_2:]
	return train, validation, test


In [58]:
class CustomDataLoader(tf.keras.utils.Sequence):

        class_dict = {"fridge" : 0, "washer dryer" : 1, "kettle" : 2, "dish washer" : 3, "microwave" : 4}

        def __init__(self, profile_path, d_type, batch_size=256, seq_len=100, denoise=True, *args, **kwargs):
                super(CustomDataLoader, self).__init__(*args, **kwargs)

                self.profile_path = Path(profile_path)
                self.batch_size = batch_size
                self.seq_len = seq_len
                self.d_type = d_type 
                self.denoise = denoise
                self.data, self.labels = self.__load()
                self.indices = np.arange(self.data.shape[0])

        def __load(self):

                if self.denoise:
                        x = np.load(self.profile_path / "training" / "denoise_inputs.npy")
                else:
                        x = np.load(self.profile_path / "training" / "noise_inputs.npy")
                y = np.load(self.profile_path / "training" / "targets.npy")
                z = np.load(self.profile_path / "training" / "states.npy")

                train_x, val_x, test_x = split_data(x)
                train_y, val_y, test_y = split_data(y)
                train_z, val_z, test_z = split_data(z)          

                if self.d_type == "train":
                        x = train_x
                        y = train_y
                        z = train_z
                elif self.d_type == "test":
                        x = test_x
                        y = test_y
                        z = test_z
                else:
                        x = val_x
                        y = val_y
                        z = val_z

                return x, (z, y)

        def __len__(self):
                return (self.data.shape[0] - self.seq_len) // self.batch_size

        def get_sample(self, index):
                indices = self.indices[index : index + self.seq_len]
                inds_inputs = sorted(indices[:self.seq_len])
                inds_labels = sorted(indices[self.seq_len-1:self.seq_len])
		
                states = self.labels[0]
                power = self.labels[1]
        
                return self.data[inds_inputs], (states[inds_labels], power[inds_labels])

        def __getitem__(self, index):
                batch = [self.get_sample(idx) for idx in range(index * self.batch_size,(index + 1) * self.batch_size)]
                batch_data, batch_label = zip(*batch)
                
                batch_data = np.array(batch_data)
                batch_label = np.array(batch_label)

                batch_label = tf.squeeze(batch_label, axis=2)

                return batch_data, tuple([batch_label[:,0,:], batch_label[:,1,:]])

In [59]:
seq_len = 100
denoise = True
dloader = CustomDataLoader(selected_profile, "train", 256, seq_len=seq_len)


In [81]:
def create_model(input_window_length):

    """Specifies the structure of a seq2point model using Keras' functional API.

    Returns:
    model (tensorflow.keras.Model): The uncompiled seq2point model.

    """
    input_layer = tf.keras.layers.Input(shape=(input_window_length,))
    reshape_layer = tf.keras.layers.Reshape((1, input_window_length, 1))(input_layer)
    conv_layer_1 = tf.keras.layers.Convolution2D(filters=30, kernel_size=(10, 1), strides=(1, 1), padding="same", activation="relu")(reshape_layer)
    conv_layer_2 = tf.keras.layers.Convolution2D(filters=30, kernel_size=(8, 1), strides=(1, 1), padding="same", activation="relu")(conv_layer_1)
    conv_layer_3 = tf.keras.layers.Convolution2D(filters=40, kernel_size=(6, 1), strides=(1, 1), padding="same", activation="relu")(conv_layer_2)
    conv_layer_4 = tf.keras.layers.Convolution2D(filters=50, kernel_size=(5, 1), strides=(1, 1), padding="same", activation="relu")(conv_layer_3)
    conv_layer_5 = tf.keras.layers.Convolution2D(filters=50, kernel_size=(5, 1), strides=(1, 1), padding="same", activation="relu")(conv_layer_4)
    flatten_layer = tf.keras.layers.Flatten()(conv_layer_5)
    label_layer = tf.keras.layers.Dense(1024, activation="relu")(flatten_layer)
    output_layer_1 = tf.keras.layers.Dense(2*5, activation="linear", name="y1_output")(label_layer)
    output_layer_2 = tf.keras.layers.Dense(5*5, activation="linear", name="y2_output")(label_layer)

    model = tf.keras.Model(inputs=input_layer, outputs=[output_layer_1, output_layer_2])
    return model

In [82]:
mod = create_model(input_window_length=seq_len)

mod.summary()

Model: "functional_13"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_6       │ (None, 100)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_6 (Reshape) │ (None, 1, 100, 1) │          0 │ input_layer_6[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_30 (Conv2D)  │ (None, 1, 100,    │        330 │ reshape_6[0][0]   │
│                     │ 30)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_31 (Conv2D)  │ (None, 1, 100,    │      7,230 │ conv2d_30[0][0]   │
│                     │ 30)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_32 (Conv2D)  │ (None, 1, 100,    │      7,240 │ conv2d_31[0][0]   │
│                     │ 40)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_33 (Conv2D)  │ (None, 1, 100,    │     10,050 │ conv2d_32[0][0]   │
│                     │ 50)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_34 (Conv2D)  │ (None, 1, 100,    │     12,550 │ conv2d_33[0][0]   │
│                     │ 50)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_6 (Flatten) │ (None, 5000)      │          0 │ conv2d_34[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 1024)      │  5,121,024 │ flatten_6[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ y1_output (Dense)   │ (None, 10)        │     10,250 │ dense_8[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ y2_output (Dense)   │ (None, 25)        │     25,625 │ dense_8[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 5,194,299 (19.81 MB)

 Trainable params: 5,194,299 (19.81 MB)

 Non-trainable params: 0 (0.00 B)

In [83]:
input_window_length = 100

In [147]:
class MultiActivationLoss(tf.keras.Loss):
        def __init__(self, *args, **kwargs):
                super().__init__(*args, **kwargs)

                self.states_loss = tf.keras.losses.SparseCategoricalCrossentropy()

        def call(self, y_true, y_pred):

                pred_state = tf.reshape(y_pred, [-1, 5, 2])

                pred_state_softmax = tf.nn.softmax(pred_state, axis=1)

                loss_nll = self.states_loss(y_true, pred_state_softmax)

                return loss_nll

In [178]:
class QuantileLoss(tf.keras.Loss):
        def __init__(self, quantiles=[0.0025,0.1, 0.5, 0.9, 0.975], *args, **kwargs):
                super().__init__(*args, **kwargs)
                self.quantiles = quantiles

        def call(self, y_true, y_pred):
                pred_rms = tf.reshape(y_pred, [-1, 5, 5])

                targets = tf.expand_dims(y_true, axis=1)
                targets = tf.repeat(targets, repeats=[5], axis=1)

                quantiles = tf.convert_to_tensor(self.quantiles, dtype=tf.float32)
                
                error = tf.transpose((targets - pred_rms), perm=[0,2,1])
                loss = tf.math.maximum(quantiles*error, (quantiles-1)*error)
                return tf.reduce_mean(loss, axis=1)

In [179]:
mod.compile(optimizer=tf.keras.optimizers.Adam(),
              loss={'y1_output' : MultiActivationLoss, 'y2_output' : QuantileLoss})

In [180]:
mod.fit(dloader, epochs=1)

2577/2577 ━━━━━━━━━━━━━━━━━━━━ 144s 55ms/step - loss: 0.6200
